<h1>Numerical Methods in Matlab/Octave</h1>

This Jupyter notebook serves as supplementary material to the code from the book [Numerical Methods for Scientific Computing](https://www.equalsharepress.com/media/NMFSC.pdf). This notebook was written and tested using Octave version 6.4.0. For information/instructions on setting up an Octave kernel in Jupyter, see the [Calysto Project](https://github.com/Calysto/octave_kernel). By and large, the snippets are verbatim from the book with an occasional explicit  output, plot statement, or variable declaration. These code snippets are minimal working toy algorithms meant to better understand the mathematics that goes into them. They are tools for tinkering and learning. Play with them and have fun. And, perhaps you can repurpose a few of them. The notebook is designed to be nonlinear⁠—you can jump around. We'll use the following URL and function throughout this notebook:

In [ ]:
bucket = "https://raw.githubusercontent.com/nmfsc/data/master/";
rgb2gray = @(x) 0.2989*x(:,:,1) + 0.5870*x(:,:,2) + 0.1140*x(:,:,3);

We can control the plotting in Jupyter by using the magic command

In [ ]:
%plot inline:qt -r 100 -f png

which sets the OpenGL graphics toolkit, resolution, and format.  (You can find other options using the magic `%plot ?` command.) Octave in Jupyter can be finicky. If plotting or Octave kernel is not working, try the `fltk` toolkit instead of `qt` or check troubleshooting hints from [Calysto Project](https://github.com/Calysto/octave_kernel). Plots in Octave in a Jupyter notebook can be rendered as an SVG or PNG. A PNG is robust but often lacks the crispness one might want for a publication-quality plot. Graphics rendered as SVGs can be later manipulated using open-source software such as Inkscape and converted to PDFs. Plotting an SVG takes longer to render and may create a rather large file size for complex plots.  Some commands such as `area` can also be problematic.  Furthermore, some systems may not handle SVGs. According to [Octave documentation](https://octave.org/doc/v7.1.0/Jupyter-Notebooks.html), if SVG images don't appear in the Jupyter notebook, you may need to explicitly trust the document (selecting "Trust Document" under the File menu).

I tend to prefer a slightly thicker line width and a slightly larger font size than the default. We can change the global settings by running the following lines:

In [ ]:
set(0, 'DefaultLineLineWidth', 2)
set(0, 'DefaultAxesFontSize', 10)
set(0, "DefaultLineMarkerSize", 16)

<h1>Notebook Contents</h1>

 [Part 1: Numerical Linear Algebra](#label0)<br>
&emsp; [Chapter 1: A Review of Linear Algebra](#label1)<br>
&emsp; [Chapter 2: Direct Methods for Linear Systems](#label2)<br>
&emsp; [Chapter 3: Inconsistent Systems](#label3)<br>
&emsp; [Chapter 4: Computing Eigenvalues](#label4)<br>
&emsp; [Chapter 6: Fast Fourier Transform](#label5)<br>
 [Part 2: Numerical Methods for Analysis](#label6)<br>
&emsp; [Chapter 7: Preliminaries](#label7)<br>
&emsp; [Chapter 8: Solutions to Nonlinear Equations](#label8)<br>
&emsp; [Chapter 9: Interpolation](#label9)<br>
&emsp; [Chapter 10: Approximating Functions](#label10)<br>
&emsp; [Chapter 11: Differentiation and Integration](#label11)<br>
 [Part 3: Numerical Differential Equations](#label12)<br>
&emsp; [Chapter 12: Ordinary Differential Equations](#label13)<br>
&emsp; [Chapter 13: Parabolic Equations](#label14)<br>
&emsp; [Chapter 16: Fourier Spectral Methods](#label15)<br>
 [Part 4: Solutions](#label16)<br>
&emsp; [Numerical Linear Algebra](#label17)<br>
&emsp; [Numerical Analysis](#label18)<br>
&emsp; [Numerical Differential Equations](#label19)<br>


<a name="label0"></a>
# Part 1: Numerical Linear Algebra
<a name="label1"></a>
## Chapter 1: A Review of Linear Algebra
**The Hilbert matrix.** The Hilbert matrix $\mathbf{H}$ is ill-conditioned even for relatively small dimensions. Taking $\mathbf{H}^{-1}\mathbf{H}$ should give us the identity matrix. 

In [ ]:
n = [10,15,20,25,50];
set(gcf,'position',[0,0,1000,200])
for i = 1:5,
  subplot(1,5,i)
  imshow(1-abs(hilb(n(i))\hilb(n(i))),[0 1])
end

<a name="label2"></a>
## Chapter 2: Direct Methods for Linear Systems
**Gaussian elimination.** The following function implements a naïve Gaussian elimination algorithm for a matrix `A` and vector `b`. We'll verify the code using a random matrix-vector pair. 

In [ ]:
function b = gaussian_elimination(A,b)
  n = length(A);
  for j=1:n
    A(j+1:n,j) = A(j+1:n,j)/A(j,j);
    A(j+1:n,j+1:n) = A(j+1:n,j+1:n) - A(j+1:n,j).*A(j,j+1:n);
  end
  for i=2:n
    b(i) = b(i) - A(i,1:i-1)*b(1:i-1);
  end
  for i=n:-1:1
    b(i) = ( b(i) - A(i,i+1:n)*b(i+1:n) )/A(i,i);
  end
end

In [ ]:
A = rand(8); b = rand(8,1);
[A\b gaussian_elimination(A,b)]

**Simplex method.** <a name="simplex"></a>The following three functions (`get_pivot`, `row_reduce`, and `simplex`) implement a naïve simplex method. Let's use them to solve the LP problem "Find the maximum of the objective function $2x + y + z$ subject to the constraints $2x+ z  \leq 3$, $4x+y + 2z  \leq 2$, and $x+y \leq 1$" along with the dual LP problem. 

In [ ]:
function [tableau] = row_reduce(tableau)
  [i,j] = get_pivot(tableau);
  G = tableau(i,:)./tableau(i,j);
  tableau = tableau - tableau(:,j)*G;
  tableau(i,:) = G;
end

In [ ]:
function [i,j] = get_pivot(tableau)
  [_,j] = max(tableau(end,1:end-1));
  a = tableau(1:end-1,j); b = tableau(1:end-1,end); 
  k = find(a > 0);
  [_,i] = min(b(k)./a(k));
  i = k(i);
end

In [ ]:
function [z,x,y] = simplex(c,A,b)
  [m,n] = size(A);
  tableau = [A eye(m) b; c' zeros(1,m) 0];
  while (any(tableau(end,1:n)>0)), 
    tableau = row_reduce(tableau);
  end
  p = find(tableau(end,1:n)==0);
  x = zeros(n,1);
  x(p) = tableau(:,p)'*tableau(:,end);
  z = -tableau(end,end);
  y = -tableau(end,n+(1:m));
end

In [ ]:
A = [2 0 1;4 1 2;1 1 0];  c = [2;1;1];  b = [3;2;1];
[z,x,y] = simplex(c,A,b)

In [ ]:
A = sprand(60,80,0.2); nnz(A), spy(A,'.')

**Graph drawing.** The following code draws the dolphin networks of the Doubtful Sound. <a name="dolphins_graph"></a>

In [ ]:
function f = spring(A,z)
  n = length(z); k = 2*sqrt(1/n);
  d = z - z.'; D = abs(d)/k;
  F = -(A.*D - 1./(D+eye(n)).^2);
  f = sum(F.*d,2);
end
function xy = spring_layout(A,z)
  n = size(A,1); 
  z = rand(n,1) + 1i*rand(n,1);
  [t,z] = ode45(@(t,z) spring(A,z),[0,10],z);
  xy = [real(z(end,:));imag(z(end,:))]';
end

In [ ]:
function xy = spectral_layout(A)
  D = diag(sum(A,2));
  [v,d] = eig(D - A);
  [_,i] = sort(diag(d));
  xy = v(:,i(2:3));
end

In [ ]:
function xy = circular_layout(A)
  n = size(A,1); t = (2*pi*(1:n)/n)';
  xy = [cos(t) sin(t)];
end

In [ ]:
function M = get_adjacency_matrix(bucket,filename)
  data = urlread([bucket filename '.csv']);
  ij = cell2mat(textscan(data,'%d,%d'));
  M = sparse(ij(:,1),ij(:,2),1);
end

In [ ]:
A = get_adjacency_matrix(bucket,"dolphins");
gplot(A,spring_layout(A),'.-'); axis equal; axis off;

In [ ]:
gplot(A,spectral_layout(A),'.-'); axis equal; axis off;

In [ ]:
gplot(A,circular_layout(A),'.-'); axis equal; axis off;

**Revised simplex method.** 

In [ ]:
function [z,x,y] = revised_simplex(c,A,b)
  [m,n] = size(A);
  N = 1:n; B = n + (1:m);
  A = [A speye(m)];
  ABinv = speye(m);
  c = [c;zeros(m,1)];
  while true
    y = c(B)'*ABinv;
    if isempty(j=find(c(N)'-y*A(:,N)>0,1)), break; end
    k = find((q = ABinv*A(:,N(j))) > 0);
    [_,i] = min(ABinv(k,:)*b./q(k));
    i = k(i);
    p = B(i); B(i) = N(j); N(j) = p;
    ABinv = ABinv - ((q - sparse(i,1,1,m,1))/q(i))*ABinv(i,:);
  end
  i = find(B<=n);
  x = zeros(n,1);
  x(B(i)) = ABinv(i,:)*b;
  z = c(1:n)'*x;
end

In [ ]:
A = [2 0 1;4 1 2;1 1 0];  c = [2;1;1];  b = [3;2;1];
[z,x,y] = revised_simplex(c,A,b)

<a name="label3"></a>
## Chapter 3: Inconsistent Systems
**Zipf's law.**  Let's use ordinary least squares to find Zipf's law coefficients for the canon of Sherlock Holmes.

In [ ]:
data = urlread([bucket 'sherlock.csv']);
T = cell2mat(textscan(data,'%s\t%d')(2));
n = length(T);
A = [ones(n,1) log(1:n)'];
B = log(T);
c1 = A\B

**Constrained least squares.** The constrained least squares problem of solving $\mathbf{Ax} = \mathbf{b}$ with the constraint condition $\mathbf{Cx}=\mathbf{d}$:

In [ ]:
function x = constrained_lstsq(A,b,C,d)
  x = [A'*A C'; C zeros(size(C,1))]\([A'*b;d])
  x = x(1:size(A,2))
end

**Total least squares.**  The function `tls`<a name="tls"></a> solves the total least squares problem.

In [ ]:
function X = tls(A,B)
  n = size(A,2);
  [_,_,V] = svd([A B],0);
  X = -V(1:n,n+1:end)/V(n+1:end,n+1:end);
end

In [ ]:
A = [2 4; 1 -1; 3 1; 4 -8]; b = [1; 1; 4; 1];
x_ols = A\b
x_tls = tls(A,b)

**Image compression.** Let's use singular value decomposition to compress an image. We'll choose a nominal rank `k = 20` for demonstration. We'll use the Frobenius norm to compute the total pixelwise error in the compressed image. Then, we'll plot out all the singular values for comparison.

In [ ]:
A = rgb2gray(imread([bucket "laura.png"]));
[U, S, V] = svd(A,0);
sigma = diag(S);

In [ ]:
k = 20;
Ak = U(:,1:k) * S(1:k,1:k) * V(:,1:k)';
norm(double(A)-Ak,'fro') - norm(sigma(k+1:end))
imshow([A,Ak])

In [ ]:
r = sum(size(A))/prod(size(A))*(1:min(size(A)));
error = 1 - sqrt(cumsum(sigma.^2))/norm(sigma); 
semilogx(r,error,'.-');

**Nonnegative matrix factorization.** The following code is a naive implementation of nonnegative matrix factorization using multiplicative updates (without a stopping criterion):

In [ ]:
function [W,H] = nmf(X,p)
  W = rand(size(X,1),p);
  H = rand(p,size(X,2));
  for i=1:50,
    W = W.*(X*H')./(W*(H*H') + (W==0));
    H = H.*(W'*X)./((W'*W)*H + (H==0));
  end
end

<a name="label4"></a>
## Chapter 4: Computing Eigenvalues
**Eigenvalue condition number.** The function `condeig` computes the eigenvalue condition number. Let's use it on a small random matrix.

In [ ]:
A = rand(4);
condeig(A)

**PageRank.** The following minimal code computes the PageRank of the very  small graph by using the power method over 9 iterations <img src="https://raw.githubusercontent.com/nmfsc/data/master/internet_graph.svg" alt="internet graph" title="internet graph" />

In [ ]:
H = [0 0 0 0 1; 1 0 0 0 0; 1 0 0 0 1; 1 0 1 0 0; 0 0 1 1 0];
v = ~any(H); 
H = H./(sum(H)+v);
n = length(H); 
d = 0.85;
x = ones([n 1])/n;
for i = 1:9
  x = d*(H*x) + d/n*(v*x)  + (1-d)/n;
end
x

<a name="label5"></a>
## Chapter 6: Fast Fourier Transform
**Radix-2 FFT.** This chapter introduces several naive functions. The radix-2 FFT algorithm is written as a recursive function `fftx2` and the inverse FFT is written as `ifftx2`.<a name="radix2fft"></a>

In [ ]:
function y = fftx2(c)
  n = length(c);
  omega = exp(-2i*pi/n); 
  if mod(n,2) == 0   
    k = (0:n/2-1)';
    u = fftx2(c(1:2:n-1));
    v = (omega.^k).*fftx2(c(2:2:n)); 
    y = [u+v; u-v];
  else   
    k = (0:n-1)';
    F = omega.^(k*k');
    y = F*c;
  end
end

In [ ]:
function c = ifftx2(y)
  c = conj(fftx2(conj(y)))/length(y);
end

**Fast Toeplitz multiplication.** The function `fasttoeplitz` computes the Toeplitz multiplication by padding out a Toeplitz matrix with zeros to make it circulant.

In [ ]:
function y = fasttoeplitz(c,r,x)
  n = length(x);
  m = 2^(ceil(log2(n)))-n;
  x1 = [c; zeros([m 1]); r(end:-1:2)];
  x2 = [x; zeros([m+n-1 1])];
  y = ifftx2(fftx2(x1).*fftx2(x2));
  y = y(1:n);
end

**Bluestein algorithm.** The following function implements  the Bluestein algorithm using fast Toeplitz multiplication.

In [ ]:
function y = bluestein(x)
  n = length(x);
  w = exp((1i*pi/n)*((0:n-1).^2))';
  y = w.*fasttoeplitz(conj(w),conj(w),w.*x);
end

**Fast Poisson solver.** The following code solves the Poisson equation using a naive fast Poisson solver and then compares the solution with the exact solution.

In [ ]:
function a = dst3(a)
  a = dstx3(shiftdim(a,1));
  a = dstx3(shiftdim(a,1));
  a = dstx3(shiftdim(a,1));
end

In [ ]:
function a = dstx3(a)
  n = size(a); o = zeros(1,n(2),n(3));
  y = [o;a;o;-a(end:-1:1,:,:)];
  y = fft(y);
  a = imag(y(2:n(1)+1,:,:)*(sqrt(2*(n(1)+1))));
end

In [ ]:
n = 50; x = (1:n)'/(n+1); dx = 1/(n+1); 
[x,y,z] = meshgrid(x);
v = @(k) 2 - 2*cos(k*pi/(n+1)); 
[ix,iy,iz] = meshgrid(1:n);
lambda = (v(ix)+v(iy)+v(iz))./dx^2;
f = 2*((x-x.^2).*(y-y.^2) + (x-x.^2).*(z-z.^2) + (y-y.^2).*(z-z.^2));
u = dst3(dst3(f)./lambda);

**DCT image compression.**

In [ ]:
pkg load signal
function [B,A] = dctcompress(A,d)
  B = dct2(A);
  idx = floor(d*prod(size(A)));
  tol = sort(abs(B(:)),'descend')(idx);
  B(abs(B)<tol) = 0;
  A = idct2(B); B = sparse(B); 
end

In [ ]:
A = rgb2gray(imread([bucket "laura.png"]));
[_,A0] = dctcompress(A,0.01);
imshow([A,A0])

<a name="label6"></a>
# Part 2: Numerical Methods for Analysis
<a name="label7"></a>
## Chapter 7: Preliminaries
Let's start with a function that returns a double-precision floating-point representation as a string of bits.

In [ ]:
function b = float_to_bin(x)
  b = sprintf("%s",dec2bin(hex2dec(num2cell(num2hex(x))),4)');
  if x<0, b(1) = '1'; end
end
float_to_bin(pi)

**Fast inverse square root.**  The following function implements the circa 1999 Q_rsqrt algorithm to approximate the reciprocal square root of a number.

In [ ]:
function y = Q_rsqrt(x)
  i = typecast(single(x),'int32');
  i = 0x5f3759df - bitshift(i,-1);
  y = typecast(i,'single');
  y = y * (1.5 - (0.5 * x * y * y));
end

In [ ]:
Q_rsqrt(0.01)

**Rump's catastrophic cancellation.**  The answer should be `-0.827396...` What does Octave come up with?

In [ ]:
a = 77617; b = 33096;
333.75*b^6+a^2*(11*a^2*b^2-b^6-121*b^4-2)+5.5*b^8+a/(2*b)

<a name="label8"></a>
## Chapter 8: Solutions to Nonlinear Equations
We start with simple implementation of the bisection method.

In [ ]:
function x = bisection(f,a,b,tolerance)
  while abs(b-a) > tolerance
    c = (a+b)/2;
    if sign(f(c)) == sign(f(a)), a = c; else b = c; end
  end
  x = (a+b)/2;
end

In [ ]:
bisection(@(x) sin(x),2,4,1e-14)

**The Mandelbrot set.** The following function takes the array `bb` for the lower-left and upper-right corners of the bounding box, `xn` for the number of horizontal pixels, and `n` for the maximum number of iterations. The function returns a two-dimensional array `M` that counts the number of iterations `k` to escape $\{z\in\mathbb{C} \mid |z^{(k)}|>2\}$.

In [ ]:
function M = escape(n,z,c)
  M = zeros(size(c));
  for k = 0:n
    mask = abs(z)<2;
    M(mask) = M(mask) + 1;
    z(mask) = z(mask).^2 + c(mask);
  end
end

In [ ]:
function M = mandelbrot(bb,xn,n,z)
  yn = round(xn*(bb(4)-bb(2))/(bb(3)-(bb(1))));
  z = z*ones(yn,xn);
  c = linspace(bb(1),bb(3),xn) + 1i*linspace(bb(4),bb(2),yn)';
  M = escape(n,z,c);
end

In [ ]:
M = mandelbrot([-0.1710,1.0228,-0.1494,1.0443],800,200,0);
%imwrite(1-M/max(M(:)),'mandelbrot.png');
imshow(1-M/max(M(:)))

Imaging the Julia set uses almost identical code. The Mandelbrot set lives in the $c$-domain with an initial value $z=0$, and the Julia set lives in the $z$-domain with a given value $c$. So the code for the Julia set requires only swapping the variables `z` and `c`.

In [ ]:
function M = julia(bb,xn,n,z)
  yn = round(xn*(bb(4)-bb(2))/(bb(3)-(bb(1))));
  z = z*ones(yn,xn);
  c = linspace(bb(1),bb(3),xn) + 1i*linspace(bb(4),bb(2),yn)';
  M = escape(n,c,z);
end

In [ ]:
J = julia([-2 -1 2 1],800,100,-1+0.3i);
%imwrite(1-J/max(J(:)),'julia.png');
imshow(1-J/max(J(:)))

**Homotopy continuation.** The following snippet of code finds a root of $$x^3-3xy^2-1 =0$$
$$y^3-3x^2y = 0$$ with an initial guess $(x,y) = (1,1)$ using homotopy continuation: 

In [ ]:
f = @(x) [x(1)^3-3*x(1)*x(2)^2-1; x(2)^3-3*x(1)^2*x(2)];
df = @(t,x,p) -[3*x(1)^2-3*x(2)^2, -6*x(1)*x(2);...
   -6*x(1)*x(2), 3*x(2)^2-3*x(1)^2]\p;
x0 = [1;1];
[t,y] = ode45(@(t,x) df(t,x,f(x0)),[0;1],x0);
y(end,:)

**Gradient descent.** The following code uses the gradient descent method to find the minimum of the Rosenbrock function and plot the intermediate steps.

In [ ]:
function plot_trace(s)
  f = @(x,y) (1-x).^2 + 100*(y' - x.^2).^2;
  x = linspace(-2,1,100);
  y = linspace(-0.25,3.25,100);
  contour(x,y, cbrt(f(x,y)), 'r'); hold on
  plot(s(:,1),s(:,2),'k.-')
end

In [ ]:
df = @(x) [-2*(1-x(1))-400*x(1)*(x(2)-x(1)^2), 200*(x(2)-x(1)^2)];
x = [-1.8,3.0]; p = [0,0]; a =  0.001; b = 0.0;
s = x;
for i = 1:500
  p = -df(x) + b*p;
  x = x + a*p;
  s = [s;x];
end
plot_trace(s)

In [ ]:
pkg load optim
f = @(x) (1-x(1)).^2 + 100*(x(2) - x(1)).^2;
x0 = [-1.8,3.0];
fminunc(f, x0)

<a name="label9"></a>
## Chapter 9: Interpolation
 **Splines.** The function `spline_natural` computes the coefficients `m` of a cubic spline with natural boundary conditions through the nodes given by the arrays `x` and `y`. The function `evaluate_spline` returns a set of `n` points along the spline. The final code snippet tests these functions with several randomly selected points.<a name="spline_natural"></a>

In [ ]:
function m = spline_natural(x,y)
  h = diff(x(:));
  gamma = 6*diff(diff(y(:))./h);
  alpha = diag(h(2:end-1),-1);
  beta = 2*diag(h(1:end-1)+h(2:end),0);
  m = [0;(alpha+beta+alpha')\gamma;0];
end

In [ ]:
function [X,Y] = evaluate_spline(x,y,m,n)
  x = x(:); y = y(:); h = diff(x); 
  B = y(1:end-1) - m(1:end-1).*h.^2/6;
  A = diff(y)./h-h./6.*diff(m);
  X = linspace(min(x),max(x),n+1)';
  i = sum(x<=X');
  i(end) = length(x)-1; 
  Y = (m(i).*(x(i+1)-X).^3 + m(i+1).*(X-x(i)).^3)./(6*h(i)) ...
      + A(i).*(X-x(i)) + B(i);
end

In [ ]:
x = linspace(0,1,8); y = rand(8,1)';
m = spline_natural(x,y);
[X,Y] = evaluate_spline(x,y,m,100);
plot(x,y,'.',X,Y)

**Bézier curves.** The following function builds a Bernstein matrix. We'll then test the function on a set of points to create a cubic Bézier curve with a set of four randomly selected control points. 

In [ ]:
B = @(n,t)[1 cumprod((n:-1:1)./(1:n))].*t.^(0:n).*(1-t).^(n:-1:0);

In [ ]:
n = 3;
t = linspace(0,1,50)';
p = rand(n+1,2);
z = B(n,t)*p;
plot(p(:,1),p(:,2),'.-',z(:,1),z(:,2))

<a name="label10"></a>
## Chapter 10: Approximating Functions
 **Legendre polynomials.** We can evaluate a Legendre polynomial of order $n$ using Bonnet's recursion formula.

In [ ]:
function P = legendre(x,n)
  if n==0,  P = ones(size(x));
    elseif n==1, P = x;
    else P = x.*legendre(x,n-1)-1/(4-1/(n-1)^2).*legendre(x,n-2);
  end
end

In [ ]:
x = linspace(-1,1,100);
for n=0:4, plot(x,legendre(x,n)); hold on; end

**Chebyshev polynomials.** We'll construct a Chebyshev differentiation matrix and use the matrix to solve a few simple problems.

In [ ]:
function [D,x] = chebdiff(n)
  x = -cos(linspace(0,pi,n))';
  c = [2;ones(n-2,1);2].*(-1).^(0:n-1);
  D = c./c'./(x - x' + eye(n));
  D = D - diag(sum(D,2));
end

In [ ]:
n = 15;
[D,x] = chebdiff(n);
u = exp(-4*x.^2);
plot(x,D*u,'.-')
t = linspace(-1,1,200);
hold on; plot(t,-8*t.*exp(-4*t.^2)); hold off

In [ ]:
B = zeros(1,n); B(1) = 1;
plot(x,[B;D]\[2;u],'.-')

In [ ]:
n = 15; k2 = 256;
[D,x] = chebdiff(n);
L = (D^2 - k2*diag(x));
L([1,n],:) = 0; L(1,1) = 1; L(n,n) = 1; 
y = L\[2;zeros(n-2,1);1];
plot(x,y,'.-')
k32 = k2^(1/3); 
a = [airy(-k32) airy(2,-k32);airy(k32) airy(2,k32)]\[2;1];
sol = @(x) a(1)*airy(k32*x) + a(2)*airy(2,k32*x);
hold on; plot(t,sol(t)); hold off

In [ ]:
N = 6:2:60; e = [];
for n = N 
  [D,x] = chebdiff(n);
  L = D^2 - k2*diag(x);
  L([1,n],:) = 0; L(1,1) = 1; L(n,n) = 1; 
  y = L\[2;zeros(n-2,1);1];
  e = [e norm(y - sol(x),inf)];
end
semilogy(N,e,'.-')

**Wavelets.** The function `scaling` returns the scaling function (father wavelet). We can use it to generate the wavelet function (mother wavelet). As an example, we will plot the Daubechies $D_4$ with $c_k = (1+\sqrt{3},3+\sqrt{3},3-\sqrt{3},1-\sqrt{3}])/4$ and $\phi(k) = (0,1+\sqrt{3},1-\sqrt{3},0)/2$.

In [ ]:
function [x,phi] = scaling(c,z,n)
  m = length(c); L = 2^n;
  phi = zeros(2*m*L,1);
  k = (0:m-1)*L;
  phi(k+1) = z;
  for j = 1:n
    for i = 1:m*2^(j-1)
      x = (2*i-1)*2^(n-j);
      phi(x+1) = c * phi(mod(2*x-k,2*m*L)+1);
    end
  end
  x = (1:(m-1)*L)/L;
  phi = phi(1:(m-1)*L);
end

In [ ]:
c = [1+sqrt(3),3+sqrt(3),3-sqrt(3),1-sqrt(3)]/4;
z = [0,1+sqrt(3),1-sqrt(3),0]/2;
[x,phi] = scaling(c,z,8);
plot(x,phi)

In [ ]:
psi = zeros(size(phi)); n = length(c)-1; L = length(phi)/(2*n)
for k = 0:n
  psi((k*L+1):(k+n)*L) += (-1)^k*c(n-k+1)*phi(1:2:end);
end
plot(x,psi)

**DWT image compression.** The following code explores using a discrete wavelet transform along with filtering as means of image compression.} %.

In [ ]:
pkg load ltfat
adjustlevels = @(x) 1 - min(max((sqrt(abs(x))),0),1);
A = rgb2gray(double(imread([bucket "laura_square.png"])))/255;
c = fwt2(A,"db2",9);
imshow(adjustlevels(c))

In [ ]:
c = thresh(c,0.5);
B = ifwt2(c,"db2",9);
imshow([A,max(min(B,1),0)]);

**Nonlinear least squares approximation.** The function `gauss_newton` solves a nonlinear least squares problem using the Levenberg–Marquardt method. The Jacobian is approximated numerically by the  function `jacobian`  using the complex-step method. The solver is then used to find the parameters for a two-Gaussian problem.<a name="jacobian"></a>

In [ ]:
function J = jacobian(f,x,c)
  for k = (n = 1:length(c))
    J(:,k) = imag(f(x,c+1e-8i*(k==n)'))/1e-8;
  end
end

In [ ]:
function c = gauss_newton(f,x,y,c)
  r = y - f(x,c);
  for j = 1:100
    G = jacobian(f,x,c);
    M = G'*G;
    c = c + (M+diag(diag(M)))\(G'*r);
    if norm(r-(r=y-f(x,c))) < 1e-10, return; end
  end
  display('Gauss-Newton did not converge.')
end

In [ ]:
f = @(x,c) c(1)*exp(-c(2).*(x-c(3)).^2) + ...
  c(4)*exp(-c(5).*(x-c(6)).^2);
x = 8*rand([100 1]);
y = f(x,[1 3 3 2 3 6]) + 0.1*randn([100 1]);
c0 = [2 0.3 2 1 0.3 7]';
c = gauss_newton(f,x,y,c0);

In [ ]:
X = linspace(0,8,100);
plot(x,y,'.',X,f(X,c));

In practice, we can use the `lsqcurvefit` function in the `optim` toolkit:

In [ ]:
pkg load optim
c = lsqcurvefit(@(c,x) f(x,c), c0, x, y)

Alternatively, we can use the  `leasqr` function in the Octave `optim` toolkit:

In [ ]:
pkg load optim
[_,c] = leasqr (x, y, c0, f)

**Logistic regression.** We'll first define the logistic function and generate synthetic data. Then, we apply Newton's method. Finally, we plot the fit logistic function.

In [ ]:
sigma = @(x) 1./(1+exp(-x));
x = rand(30,1); y = ( rand(30,1) < sigma(10*x-5) );

In [ ]:
X = [ones(size(x)) x]; w = zeros(2,1);
for i=1:10
  S = sigma(X*w).*(1 - sigma(X*w));
  w = w + (X'*(S.*X))\(X'*(y - sigma(X*w)));
end

In [ ]:
t = linspace(min(x),max(x),50);
plot(x,y,'.',t,sigma([ones(size(t))' t']*w))

**Neural Networks.** Let's use a neural network to find a function that approximates semi-circular data.

In [ ]:
N = 100; t = linspace(0,pi,N);
x = cos(t); x = [ones(1,N);x];
y = sin(t) + 0.05*randn(1,N);
n = 20; W1 = rand(n,2); W2 = randn(1,n);
phi = @(x) max(x,0);
dphi = @(x) (x>0);

In [ ]:
alpha = 1e-3;
for epoch = 1:10000
  s = W2 * phi(W1*x);
  dLdy = 2*(s-y);
  dLdW1 = dphi(W1*x) .* (W2' * dLdy) * x';   
  dLdW2 = dLdy * phi(W1*x)';
  W1 -= 0.1 * alpha * dLdW1; 
  W2 -= alpha * dLdW2; 
end

In [ ]:
s = W2 * phi(W1*x);
scatter(x(2,:),y,'r','filled'); hold on
plot(x(2,:),s)

In [ ]:
N = 100; t = linspace(0,pi,N);
x = cos(t); x = [ones(1,N);x];
y = sin(t) + 0.05*randn(1,N);
n = 20; W1 = rand(n,2); W2 = randn(1,n);
phi = @(x) 1./(1+exp(-x));
dphi = @(x) phi(x).*(1-phi(x));

alpha = 1e-1;
for epoch = 1:10000
  s = W2 * phi(W1*x);
  L = norm(s- y);
  dLdy = 2*(s-y)/L;
  dLdW1 = dphi(W1*x) .* (W2' * dLdy) * x';   
  dLdW2 = dLdy * phi(W1*x)';
  W1 -= 0.1 * alpha * dLdW1; 
  W2 -= alpha * dLdW2; 
end
s = W2 * phi(W1*x);
scatter(x(2,:),y,'r','filled'); hold on
plot(x(2,:),s)

<a name="label11"></a>
## Chapter 11: Differentiation and Integration
 Let's compute the coefficients to the third-order approximation to $f'(x)$ using nodes at $x-h$, $x$, $x+h$ and $x+2h$. 

In [ ]:
d = [-1;0;1;2];
n = length(d);
V = fliplr(vander(d)) ./ factorial([0:n-1]);
C = inv(V);
rats(C(2,:))

**Richardson extrapolation.** $D(\phi(x))$ of a finite difference operator $\phi(x)$.<a name="richardson_extrapolation"></a>

In [ ]:
function D = richardson(f,x,m,n)
  if n > 0
    D = (4^n*richardson(f,x,m,n-1) - richardson(f,x,m-1,n-1))/(4^n-1);
  else
    D =  phi(f,x,2^m);
  end
end

In [ ]:
function p = phi(f,x,n) 
  p = (f(x+1/n) - f(x-1/n))/(2/n);
end

In [ ]:
richardson(@(x) sin(x),0,4,4)

**Automatic differentiation.** <a name="dualclass"></a> We can build a minimal working example of forward accumulation automatic differentiation by defining a class and overloading the base operators. You'll first need to save the following code as an m-file called `dual.m`. I've added a Jupyter magic command to the top of the cell to do this. We'll verify the code using the function $x + \sin x$. 

In [ ]:
%%file dual.m
classdef dual
  properties
    value
    deriv
  end 
  methods
    function obj = dual(a,b)
      obj.value = a;
      obj.deriv = b;
    end
    function h = plus(u,v)
      if ~isa(u,'dual'), u = dual(u,0); end
      if ~isa(v,'dual'), v = dual(v,0); end
      h = dual(u.value + v.value, u.deriv + v.deriv);
    end
    function h = mtimes(u,v)
      if ~isa(u,'dual'), u = dual(u,0); end
      if ~isa(v,'dual'), v = dual(v,0); end
      h = dual(u.value*v.value, u.deriv*v.value + u.value*v.deriv);
    end
    function h = sin(u)
      h = dual(sin(u.value), cos(u.value)*u.deriv);
    end
    function h = minus(u,v)
      h = u + (-1)*v;
    end
  end
end

In [ ]:
x = dual(0,1);
y = x + sin(x);
[y.value,y.deriv]

Now, let's apply the code above to compute the Jacobian of the system $$y_1 = x_1x_2 + \sin x_2$$$$y_2 = x_1x_2 - \sin x_2$$ evaluated at $(x_1,x_2) = (2,\pi)$.

In [ ]:
x1 = dual(2,[1 0]);
x2 = dual(pi,[0 1]);
y1 = x1*x2 + sin(x2);
y2 = x1*x2 - sin(x2);
[y1.value,y2.value,y1.deriv,y2.deriv]

**Romberg method.** We can use the following trapezoidal quadrature  to make a Romberg method using Richardson extrapolation. We first define the function `trapezoidal` for composite trapezoidal quadrature. By redefining `phi` to be `trapezoidal`, we can simply apply the function `D` that we used to define Richardson extrapolation. We'll verify the code by integrating $\sin x$ from $0$ to $\pi/2$.

In [ ]:
function p = trapezoidal(f,x,n) 
  F = f(linspace(x(1),x(2),n+1));
  p = (F(1)/2 + sum(F(2:end-1)) + F(end)/2) * (x(2)-x(1))/n;
end
function p = phi(f,x,n) 
  p =  trapezoidal(f,x,n);
end
richardson(@(x) sin(x),[0,pi/2],4,4)

**Composite trapezoidal method.** Let's examine the convergence rate for the composite trapezoidal rule applied to the function $f(x) = x + (x - x^2)^p$ over the interval $[0,2]$ with $p = 1,2,\dots,7$. We can do this by finding the logl-og slope of the error as a function of subintervals $n$. We find that the error of the trapezoidal rule is $O(n^2)$ when $p=1$, $O(n^4)$ when $p$ is 2 or 3, $O(n^6)$ when $p$ is 4 or 5, and so on.

In [ ]:
error = [];
n = floor(logspace(1,2,10));
for p = 1:7, 
  f = @(x) x + x.^p.*(2-x).^p;
  S = trapezoidal(f,[0,2],1e6);
  for i = 1:length(n)
    Sn = trapezoidal(f,[0,2],n(i));
    error(i) = abs(Sn - S)/S;
  end
  slope(p) = [log(error)/[log(n);ones(size(n))]](1);
  loglog(n,error,'.-k'); hold on;
end

**Clenshaw–Curtis quadrature.** applies the trapezoidal rule to a discrete cosine transform (type-1) as a means of numerically evaluating the integral $\int_{-1}^{1} f(x) \,\mathrm{d}x$.  We'll test the integral on the function $f(x) = 8 \cos x^2$, with an integral of approximately 0.566566

In [ ]:
function S = clenshaw_curtis(f,n)
  x = cos(pi*(0:n)'/n);
  w = zeros(1,n+1); w(1:2:n+1) = 2 ./ (1 - (0:2:n).^2);
  S = 2/n * w * dctI(f(x));
end

In [ ]:
function a = dctI(f)
  n = length(f);
  g = real( fft( [f;f(n-1:-1:2)] ) ) / 2;
  a = [g(1)/2; g(2:n-1); g(n)/2];
end

In [ ]:
clenshaw_curtis(@(x) cos(8*x.^2),20)

A mathematical comment: we could have also  defined a type-1 DCT explicitly in terms of its underlying FFTs if we wanted to crack the black box open just a wee bit more.

**Gauss–Legendre quadrature.** We first compute the Legendre weights and nodes and then apply Gauss–Legendre quadrature to compute $$\int_{-1}^{1} \cos x \cdot \mathrm{e}^{-x^2} \,\mathrm{d}x$$ using a nominal number of nodes $n=8$. 

In [ ]:
function [nodes,weights] = gauss_legendre(n)
  b = (1:n-1).^2./(4*(1:n-1).^2-1);
  a = zeros(n,1);
  scaling = 2;
  [v,s] = eig(diag(sqrt(b),1) + diag(a) + diag(sqrt(b),-1));
  weights = scaling*v(1,:).^2;
  nodes = diag(s);
end

In [ ]:
n = 8;
f = @(x)  cos(x).*exp(-x.^2);
[nodes,weights] = gauss_legendre(n);
weights * f(nodes)

<a name="label12"></a>
# Part 3: Numerical Differential Equations
<a name="label13"></a>
## Chapter 12: Ordinary Differential Equations
 Let's plot the boundary of the region of absolute stability for BDF2: $$z = \frac{\frac{3}{2} r^2 - 2 r + \frac{1}{2}}{r^2}$$

In [ ]:
r = exp(2i*pi*(0:0.01:1));
plot((1.5*r.^2 - 2*r + 0.5)./r.^2); axis equal;

**Multistep coefficients.** The function `multistepcoefficients` determines the multistep coefficients for stencil given by `m` and `n`. The function `plotstability` uses these coefficients to plot boundary of the region of absolute stability. We'll test it on the Adams–Moulton method with input `m = [0 1]` and `n = [0 1 2]`.<a name="multistepcoefficients"></a>

In [ ]:
function [a,b] = multistepcoefficients(m,n)
  s = length(m) + length(n) - 1;
  A = (m+1).^((0:s-1)');
  B = ((0:s-1).*(n+1)'.^[0,0:s-2])';
  c = -[A(:,2:end) B]\ones(s,1);
  a = [1;c(1:length(m)-1)];
  b = [c(length(m):end)];
end

In [ ]:
function plotstability(a,b)
  r = exp(1i*linspace(.01,2*pi-0.01,400));
  z = (r'.^(1:length(a))*a) ./ (r'.^(1:length(b))*b);
  plot(z); axis equal
end

In [ ]:
m = [0,1]; n = [0,1,2];
[a,b] = multistepcoefficients(m,n);
plotstability(a,b)

**Recipe for solving an ODE.** The general recipe for solving an ODE is to
1. Define the problem
1. Set up the parameters
1. (Choose the method and) solve the problem
1. Present the solution

Let's apply this recipe to solve the pendulum problem $u'' = \sin u$ with initial conditions $u(0) = \pi/9$ and $u'(0) = 0$ over $t\in[0,8\pi]$.

In [ ]:
pendulum = @(t,u)[u(2);-sin(u(1))];
u0 = [8*pi/9,0]; tspan = [0,8*pi];
[t,u] = ode23(pendulum,tspan,u0);
plot(t,u(:,1),'.-',t,u(:,2),'.-');
legend('\theta','\omega');

<a name="label14"></a>
## Chapter 13: Parabolic Equations
**Heat equation using the backward Euler method.** Let's solve the heat equation using the backward Euler method with initial conditions given by a rectangular function and absorbing boundary conditions.

In [ ]:
dx = .01; dt = .01; L = 2; lambda = dt/dx^2; uL = 0; uR = 0;
x = (-L:dx:L)'; n = length(x); 
u = (abs(x)<1);
u(1) += 2*lambda*uL; u(n) += 2*lambda*uR; 
D = spdiags(repmat([1 -2 1],[n 1]),-1:1,n,n);
D(1,2) = 0;  D(n,n-1)= 0; 
A = speye(n) - lambda*D;
for i = 1:20
  u = A\u;
end
area(x,u,"edgecolor",[1 .5 .5],"facecolor",[1 .8 .8])

**Heat equation using the Crank–Nicolson method.** Let's solve the heat equation again using the Crank–Nicolson method with initial conditions given by a rectangular function. This time we'll use reflecting boundary conditions. Notice how the high-frequency information does not decay as it did when using the backward Euler method.

In [ ]:
dx = .01; dt = .01; L = 2; lambda = dt/dx^2;
x = (-L:dx:L)'; n = length(x); 
u = (abs(x)<1);
D = spdiags(repmat([1 -2 1],[n 1]),-1:1,n,n);
D(1,2) = 2;  D(n,n-1) = 2; 
A = 2*speye(n) + lambda*D; 
B = 2*speye(n) - lambda*D; 
for i = 1:20
  u = B\(A*u);
end
plot(x,u);

**Porous medium equation.** We'll now solve the porous medium equation $u_t = (u^2u_x)_x$ using an adaptive-step BDF routine.

In [ ]:
n = 400; L = 2; x = linspace(-L,L,n)'; dx = x(2)-x(1);
m = @(u) u.^2;
Du = @(t,u) [0;diff(m((u(1:n-1)+u(2:n))/2).*diff(u))/dx^2;0];
u0 = double(abs(x)<1);
options = odeset('JPattern',spdiags(ones([n 3]),-1:1,n,n));
[t,U] = ode15s(Du,linspace(0,2,10),u0,options); 
for i=1:size(U,1), plot(x,U(i,:),'r'); hold on; ylim([0 1]); end

<a name="label15"></a>
## Chapter 16: Fourier Spectral Methods
**Heat equation.** The formal solution to the heat equation is $$u(t,x) = \mathrm{F}^{-1}\left[  \mathrm{e}^{-\xi^2 t}  \mathrm{F} u(0,x) \right].$$

In [ ]:
n = 256; L = 4;
k2 = ([0:floor(n/2) floor(-n/2+1):-1]*(2*pi/L)).^2;
u = @(t,u0) real(ifft(exp(-k2*t).*fft(u0)))

In [ ]:
x = L*(1:n)/n-L/2;
u0 = (abs(x)<1);
plot(x,u(0.1,u0));

**Navier–Stokes equation.** We solve the Navier–Stokes equation in three parts: define the functions, initialize the variables, and iterate over time.

In [ ]:
flux = @(Q,c) c.*diff([Q(end,:);Q]) ...
  + 0.5*c.*(1-c).*diff([Q(end,:);Q;Q(1,:)],2); 
H = @(u,v,ikx,iky) fft2(ifft2(u).*ifft2(ikx.*u) ...
    + ifft2(v).*ifft2(iky.*u));

In [ ]:
L = 2; n = 128; e = 0.001; dt = .001; dx = L/n;
x = linspace(dx,L,n); y = x';
Q = 0.5*(1+tanh(10*(1-abs(L/2 -y)/(L/4)))).*ones(size(x));
u = Q.*(1+0.5*sin(L*pi*x));
v = zeros(size(u)); 
u = fft2(u); v = fft2(v); 
us = u; vs = v;
ikx = 1i*[0:n/2 (-n/2+1):-1]*(2*pi/L);
iky = ikx.';
k2 = ikx.^2+iky.^2;
Hx = H(u,v,ikx,iky); Hy = H(v,u,iky,ikx);
M1 = 1/dt + (e/2)*k2;
M2 = 1/dt - (e/2)*k2;

In [ ]:
for i = 1:1200
  Q = Q - flux(Q,(dt/dx)*real(ifft2(v))) ...
       - flux(Q',(dt/dx)*real(ifft2(u))')';
  Hxo = Hx;  Hyo = Hy;  
  Hx = H(u,v,ikx,iky);  Hy = H(v,u,iky,ikx);
  us = u - us + (-1.5*Hx + 0.5*Hxo + M1.*u)./M2;
  vs = v - vs + (-1.5*Hy + 0.5*Hyo + M1.*v)./M2;
  phi = (ikx.*us + iky.*vs)./(k2+(k2==0));  
  u = us - ikx.*phi;
  v = vs - iky.*phi;
end
contourf(x,y,Q,[.5 .5]); axis equal;

<a name="label16"></a>
# Part 4: Solutions
<a name="label17"></a>
## Numerical Linear Algebra
**1.4. Invertibility of random (0,1) matrices.** The number of invertible $n\times n$ (0,1) matrices is known for $n$ up to 8. (See the [On-Line Encyclopedia of Integer Sequences](http://oeis.org/A055165).) We'll approximate the ratio of invertible matrices by checking the determinants of randomly drawn ones. 

In [ ]:
N = 10000; n = zeros([1 20]);
for d = 1:20
  for j = 1:N
    n(d) = n(d) + (det(rand(d)>0.5)~=0);
  end
end
plot(1:20,n/N,'.-')

**2.3. Naive algorithm for the determinant.** The determinant of  matrix $\mathbf{A}$ is given by the product of the elements along the diagonal of $\mathbf{U}$ multiplied by the parity of the permutation matrix $\mathbf{P}$ from the LU decomposition $\mathbf{PLU} = \mathbf{A}$.

In [ ]:
function D = detx(A)
  [L,U,P] = lu(A);
  s = 1;
  for i = 1:length(P)
    m = find(P(i+1:end,i));
    if m, P([i i+m],:) = P([i+m i],:); s = -1*s; end
  end
  D = s * prod(diag(U));
end

In [ ]:
A = rand(20,20);
detx(A) - det(A)

**2.4. Reverse Cuthill–McKee algorithm.** The following function implements a naive reverse Cuthill–McKee algorithm  for symmetric matrices. We'll verify the algorithm by applying it to a sparse, random (0,1) matrix.

In [ ]:
function p = rcuthillmckee(A)
  A = spones(A);
  [_,r] = sort(sum(A));
  p = [];
  while ~isempty(r)
    q = r(1); r(1) = [];
    while ~isempty(q)
      p = [p q(1)];  
      [_,k] = find(A(q(1),r));
      q = [q(2:end) r(k)]; r(k) = []; 
    end
  end
  p = fliplr(p);
end

In [ ]:
A = sprand(1000,1000,0.001); A = A + A';
p = rcuthillmckee(A);
subplot(1,2,1); spy(A,'.',2); axis equal
subplot(1,2,2); spy(A(p,p),'.',2); axis equal

**2.5. Dolphins of Doubtful Sound.**  We'll reuse the code [above](#dolphins_graph) used to draw the original graph of the dolphins.

In [ ]:
function xy = circular_layout(A)
  n = size(A,1); t = (2*pi*(1:n)/n)';
  xy = [cos(t) sin(t)];
end

In [ ]:
A = get_adjacency_matrix(bucket,"dolphins");
gplot(A,circular_layout(A),'.-')
axis equal; axis off;

In [ ]:
p = rcuthillmckee(A);
gplot(A(p,p),circular_layout(A),'.-')
axis equal; axis off;

**2.7. Stigler diet problem.** Let's solve the Stigler diet problem. We'll use the function naïve [`simplex`](#simplex) defined above.

In [ ]:
function [tableau] = row_reduce(tableau)
  [i,j] = get_pivot(tableau);
  G = tableau(i,:)./tableau(i,j);
  tableau = tableau - tableau(:,j)*G;
  tableau(i,:) = G;
end

function [i,j] = get_pivot(tableau)
  [_,j] = max(tableau(end,1:end-1));
  a = tableau(1:end-1,j); b = tableau(1:end-1,end); 
  k = find(a > 0);
  [_,i] = min(b(k)./a(k));
  i = k(i);
end

function [z,x,y] = simplex(c,A,b)
  [m,n] = size(A);
  tableau = [A eye(m) b; c' zeros(1,m) 0];
  while (any(tableau(end,1:n)>0)), 
    tableau = row_reduce(tableau);
  end
  p = find(tableau(end,1:n)==0);
  x = zeros(n,1);
  x(p) = tableau(:,p)'*tableau(:,end);
  z = -tableau(end,end);
  y = -tableau(end,n+(1:m));
end

In [ ]:
urlwrite([bucket "diet.mat"],"diet.mat");
load diet.mat
A = values'; b = minimums; c = ones(size(A,2),1);
[z,x,y] = simplex(b,A',c);
cost = z, food{find(y!=0)}

In practice, we can use Octave's built-in `glpk` function:

In [ ]:
[x,z] = glpk(c,A,b,[],[],repmat("L",size(b')));
cost = z, food{find(x!=0)}

**2.8. Six degrees of Kevin Bacon.** Let's determine the shortest path between two actors along with their connecting movies. We'll first define helper functions `get_names`<a name="get_names"></a> and `get_adjacency_matrix`<a name="get_adjacency_matrix"></a>. Then we'll build an biadjacency matrix $\mathbf{B}$ and construct the adjacency matrix $$\mathbf{A} = \begin{bmatrix} \mathbf{0} & \mathbf{B}^\mathsf{T} \\ \mathbf{B} & \mathbf{0}\end{bmatrix}.$$

In [ ]:
function r = get_names(bucket,filename)
  data = urlread([bucket filename '.txt']);
  r = cell2mat(textscan(data,'%s','delimiter', '\n')(1));
end
function M = get_adjacency_matrix(bucket,filename)
  data = urlread([bucket filename '.csv']);
  ij = cell2mat(textscan(data,'%d,%d'));
  M = sparse(ij(:,1),ij(:,2),1);
end

In [ ]:
actors = get_names(bucket,"actors");
movies = get_names(bucket,"movies");
B = get_adjacency_matrix(bucket,"actor-movie");
[m,n] = size(B);
A = [sparse(n,n) B';B sparse(m,m)];
actormovie = [actors;movies];

In [ ]:
function path = findpath(A,a,b)
 p = -ones(size(A,2),1);
 q = a; p(a) = -9999; i = 1;
 while i<=length(q),
    k = find(A(q(i),:));
    k = k(p(k)==-1);
    q = [q k];  p(k) = q(i);  i = i + 1;
    if any(k==b),
      path = backtrack(p,b);  return;
    end
  end
  display("No path.");
end

In [ ]:
function path = backtrack(p,b)
  s = b; i = p(b);
  while i != -9999, s = [s i]; i = p(i); end
  path = s(end:-1:1);
end

In [ ]:
a = find(ismember(actors,"Bruce Lee")); 
b = find(ismember(actors,"Tommy Wiseau"));
actormovie(findpath(A,a,b)){:}

**2.9. Sparse matrix storage efficiency.**

In [ ]:
d = 0.5; A = sprand(60,80,d);
s = whos('A'); nbytes = s.bytes;
nbytes, 8*(2*d*prod(size(A)) + size(A,2) + 1)

**3.4. Image deblurring.** Take `X` to grayscale image (with pixel intensity between 0 and 1), `A` and `B`  to be Gaussian blurring matrices with standard deviations of 20 pixels, and `N` to be a matrix of random values  from the uniform distribution over the interval $(0,0.01)$. We'll compare three deblurring methods: inverse, Tikhonov regulation, and the pseuodinverse. We can find a good value for regulariation parameter α = 0.05 with some trial-and-error.

In [ ]:
X = rgb2gray(double(imread([bucket "laura.png"])))/255;
[m,n] = size(X);
blur = @(x) exp(-(x/20).^2/2);
A = blur([1:m] - [1:m]'); A = A./sum(A,2);
B = blur([1:n] - [1:n]'); B = B./sum(B,1);
N = 0.01*rand(m,n);
Y = A*X*B + N;

In [ ]:
a = 0.05;
X1 = A\Y/B;
X2 =  (A'*A+a^2*eye(m))\A'*Y*B'/(B*B'+a^2*eye(n));
X3 = pinv(A,a)*Y*pinv(B,a);
imshow(max(min([X Y X1 X2 X3],1),0))

**3.5. Filippelli problem.** The National Institute for Standards and Technology (NIST) contrived the Filippelli dataset to benchmark linear regression software. The Filippelli problem consists of fitting a 10th degree polynomial to the data set⁠—a rather ill-conditioned problem. We first need to download the data. Then we'll define three methods for solving the Vandermonde problem: the normal equation, QR decomposition, and the pseudoinverse.

In [ ]:
data = urlread([bucket 'filip.csv']);
T = cell2mat(textscan(data,'%f,%f'));
y = T(:,1); x = T(:,2);
data = urlread([bucket 'filip-coeffs.csv']);
beta = cell2mat(textscan(data,'%f'));

In [ ]:
function [c,r] = solve_filip(x,y,n)
  V = vander(x, n);
  c(:,1) = (V'*V)\(V'*y);
  [Q,R] = qr(V,0);
  c(:,2) = R\(Q'*y);
  c(:,3) = pinv(V,1e-10)*y;
  for i=1:3, r(i) = norm(V*c(:,i)-y); end
end

In [ ]:
build_poly = @(c,X) vander(X,length(c))*c;

Now, let's solve the problem and plot the results. Let's also list the coefficients from each method alongside the official NIST coefficients. What do you notice about the coefficients? What methods performs the best?

In [ ]:
n = 11;
[c,r] = solve_filip(x,y,n);
X = linspace(min(x),max(x),200);
Y = build_poly(c,X);
plot(X,Y,x,y,'.');ylim([0.7,0.95])

What makes the Filippelli problem difficult is that the system's huge condition number. We can reduce the condition number by first standardizing the data before using it⁠—i.e., subtracting the mean and dividing by the standard deviation. Look at the difference in condition numbers of the Vandermonde matrix before and after standardizing the data.

In [ ]:
zscore  = @(X,x) (X .- mean(x))/std(x);
[cond(vander(x,11)) cond(vander(zscore(x,x),11))]
[c,r] = solve_filip(zscore(x,x), zscore(y,y), n);
for i=1:3, 
  Y(:,i) = build_poly(c(:,i),zscore(X,x))*std(y).+mean(y); 
end
plot(X,Y,x,y,'.');ylim([0.7,0.95])

**3.7. Modeling daily temperatures.** We'll use $u(t) = c_1 \sin(2\pi t) + c_2 \cos(2\pi t) + c_3$ to model the daily temperatures using data recorded in Washington, DC. between 1967 and 1971.

In [ ]:
data = urlread([bucket 'dailytemps.csv']);
T = textscan(data,'%s%f','HeaderLines',1,'Delimiter',',');
day = datenum(T{1},'yyyy-mm-dd'); temp = T{2};
day = (day-day(1))/365;
tempsmodel = @(t) [sin(2*pi*t) cos(2*pi*t) ones(size(t))];
c = tempsmodel(day)\temp;
plot(day,temp,'.',day,tempsmodel(day)*c,'k');

**3.8. Image recognition.** We'll practice  identifying handwritten digits using the MNIST database. Instead of using the [NIST website](https://www.nist.gov/itl/products-and-services/emnist-dataset) provides a version formatted as a Matlab MAT-file. The entire container from the NIST website is quite large (around 700MB), and we only need a smaller 20MB set. A copy of just the emnist-nist file is avilable at [link](https://github.com/nmfsc/data/raw/master/emnist-mnist.mat).

In [ ]:
k = 12; V = [];
urlwrite([bucket "emnist-mnist.mat"],"emnist-mnist.mat")
load emnist-mnist.mat
for i = 1:10
  D = dataset.train.images(dataset.train.labels==i-1,:);
  [_,_,V(:,:,i)] = svds(double(D),k); 
end

In [ ]:
pix = reshape(V(:,:,3+1),[28,28*k]);
imshow(pix,[]); axis off
set(gcf,'position',[0,0,1000,100])

In [ ]:
r = [];
d = double(dataset.test.images)';
for i = 1:10
  r(i,:) = sum((V(:,:,i)*(V(:,:,i)'*d) - d).^2);
end
[c,predicted] = min(r);

In [ ]:
for i = 1:10
  x = predicted(find(dataset.test.labels == i-1));
  confusion(i,:) = histc(x,1:10);
end  
confusion

**3.9. Actor similarity model.** We use SVD to find a lower-dimensional subspace relating actors and genres. Then we find the closest actors in that subspace using cosine similarity. We use the functions [`get_names`](#get_names) and  [`get_adjacency_matrix`](#get_adjacency_matrix) developed earlier.

In [ ]:
actors = get_names(bucket,"actors");
genres = get_names(bucket,"genres");
A = get_adjacency_matrix(bucket,"movie-genre"); A = A/diag(sum(A));
B = get_adjacency_matrix(bucket,"actor-movie");

In [ ]:
[U,S,V] = svds(A*B, 12);
Q = V'./sqrt(sum(V'.^2));
q = Q(:,find(ismember(actors,"Steve Martin"))); 
[_,r] = sort(Q'*q,'descend');
actors(r(1:10)){:}

Let's also see Steve Martin's genre signature.

In [ ]:
[p,r] = sort(U*S*q,'descend');
for i=1:10, printf("%s: %4.3f\n",genres(r(i)){:},p(i)/sum(p)), end

**3.10. Multilateration.** We use ordinary least squares and total least squares to solve a multilateration problem. The function [`tls`](#tls) is defined above.

In [ ]:
xyt = [3 3 12; 1 15 14; 10 2 13; 12 15 14; 0 11 12];
reference = xyt(1,:); xyt = xyt - reference;
A = [2 2 -2].*xyt; b = (xyt.^2)*[1; 1; -1];
x_ols = A\b + reference'
x_tls = tls(A,b) + reference'

**4.1. Girko's circular law.** Let's plot out the eigenvalues of a few thousand normal random matrices of size $n$ to get a probability distribution in the complex plane.  What do you notice?

In [ ]:
n = 8; N = 2500; E = zeros(n*N,1);
for i = 0:N-1, E(n*i+(1:n)) = eig(randn(n,n)); end
plot(E,'.'); axis equal

**4.4. Rayleigh quotient iteration.** Let's define a function that finds an eigenvalue $\lambda$ and eigenvector $\mathbf{x}$ of a matrix. The function will choose a random initial guess for $\mathbf{x}$ unless one is provided. We'll then verify the algorithm on a symmetric matrix. Rayleigh quotient iteration works on general classes of matrices, but it often has difficulty converging when matrices get large or far from symmetric⁠—i.e., when eigenvectors get closer together.

In [ ]:
function [rho,x] = rayleigh(A)
  n = length(A);  x = randn([n 1]);
  while true
    x = x/norm(x);
    rho = x'*A*x;
    M = A-rho*eye(n);
    if rcond(M)<eps, break; end
    x = M\x;
  end
end
A = [2 3 6 4; 3 0 3 1; 6 3 8 8; 4 1 8 2];
rayleigh(A)

**4.5. Implicit QR method.** We'll define a function that computes all the eigenvalues of a matrix using the implicit QR method. We'll then verify the algorithm on a matrix with known eigenvalues.

In [ ]:
function eigenvalues = implicitqr(A)
  n = length(A);
  tolerance = 1e-12;
  H = hess(A);
  while true
    if abs(H(n,n-1))<tolerance,
      n = n-1; if n<2, break; end;
    end
    Q = givens(H(1,1)-H(n,n),H(2,1));
    H(1:2,1:n) = Q*H(1:2,1:n);
    H(1:n,1:2) = H(1:n,1:2)*Q';
    for i = 2:n-1
      Q = givens(H(i,i-1),H(i+1,i-1));
      H(i:i+1,1:n) = Q*H(i:i+1,1:n);
      H(1:n,i:i+1) = H(1:n,i:i+1)*Q';
    end
  end
  eigenvalues = diag(H);
end

In [ ]:
n = 20; S = randn(n); 
D = diag(1:n); A = S*D*inv(S);
implicitqr(A)

**4.6. Hollywood eigenvector centrality.** We'll use eigenvector centrality to determine who's at the center of Hollywood. We use the functions [`get_names`](#get_names) and  [`get_adjacency_matrix`](#get_adjacency_matrix) developed earlier.

In [ ]:
actors = get_names(bucket,"actors");
B = get_adjacency_matrix(bucket,"actor-movie");
M = sparse(B'*B);
v = ones(size(M,1),1);
for k = 1:10,
  v = M*v; v /= norm(v);
end
[_,r] = sort(v,'descend');
actors(r(1:10)){:}

**4.7. Randomized SVD.** We define a method that implements randomized SVD. The idea is to start with a set of $k$ random vectors and perform a few steps of the naive QR method to generate a $k$-dimensional subspace closer to the space of dominant singular values. Then, we perform SVD on that subspace. We may not get the exact singular values, but we just need a good guess. Because the matrix is huge, this method can be significantly faster than SVD or sparse SVD.

In [ ]:
function [U,S,V] = randomizedsvd(A,k)
  Z = rand([size(A,2) k]);
  [Q,R] = qr(A*Z,0);
  for i = 1:3
    [Q,R] = qr(A'*Q,0);
    [Q,R] = qr(A*Q,0);
  end
  [W,S,V] = svd(Q'*A,0);
  U = Q*W;  
end

In [ ]:
A = double(rgb2gray(imread([bucket "red-fox.jpg"])));
[U,S,V] = randomizedsvd(A,10);
imshow([A U*S*V'])

In [ ]:
k = 10;
tic; [U,S,V] = randomizedsvd(A,k); 
t1 = toc; s1 = diag(S);
tic; s2 = svds(A,k);
t2 = toc;
tic; [U,S,V] = svd(A); 
t3 = toc; s3 = diag(S)(1:k);
[t1 t2 t3]
[s1 s2 s3]

**4.8. 100-dollar, 100-digit challenge.** "The infinite matrix $\mathbf{A}$ with entries a₁₁=1, a₁₂ = 1/2, a₂₁ = 1/3, a₁₃ = 1/4, a₂₂ = 1/5, a₃₁ = 1/6, and so on, is a bounded operator on $\ell^2$. What is $\|\mathbf{A}\|_2$?"

In [ ]:
m = 5000; j = 1:m;
A = zeros(m,m);
for i = 1:m
  A(i,j) = 1./(1 + (i+j-1).*(i+j)/2 - j);
end
c = svds(A,1)

**5.4. 3D Poisson equation.** Let's compare the Jacobi, Gauss–Seidel, SOR, and conjugate gradient methods on the Poisson equation over the unit cube.

In [ ]:
n = 50; x = (1:n)'/(n+1); dx = 1/(n+1); 
[x,y,z] = meshgrid(x);
I = speye(n);
D = spdiags(repmat([1 -2 1],[n 1]),-1:1,n,n);
A = (kron(kron(D,I),I) + kron(I,kron(D,I)) + kron(I,kron(I,D)))/dx^2; 
f = ((x-x.^2).*(y-y.^2)+(x-x.^2).*(z-z.^2)+(y-y.^2).*(z-z.^2))(:);
ue = ((x-x.^2).*(y-y.^2).*(z-z.^2)/2)(:);

In [ ]:
function e = stationary(A,b,w,n,ue)
  e = zeros(n,1); u = zeros(size(b));
  P = tril(A,0) - (1-w)*tril(A,-1);
  for i=1:n
    u = u + P\(b-A*u);
    e(i) = norm(u - ue, 1);
  end
end

In [ ]:
function e = conjugategradient(A,b,n,ue)
  e = zeros(n,1); u = zeros(size(b)); 
  r = b-A*u; p = r;
  for i=1:n
    Ap = A*p;
    a = (r'*p)/(Ap'*p);
    u = u + a.*p; r = r - a.*Ap;
    b = (r'*Ap)/(Ap'*p);
    p = r - b.*p;
    e(i) = norm(u - ue, 1);
  end
end

In [ ]:
tic; err(:,1) = stationary(A,-f,0,400,ue); toc
tic; err(:,2) = stationary(A,-f,1,400,ue); toc
tic; err(:,3) = stationary(A,-f,1.9,400,ue); toc
tic; err(:,4) =  conjugategradient(A,-f,400,ue); toc
semilogy(err); legend("Jacobi","Gauss-Seidel","SOR","Conj. Grad.")

**5.5.  100-dollar, 100-digit challenge.** "Let $\mathbf{A}$ be the 20000$\times$20000 matrix whose entries are zero everywhere except for the primes 2, 3, 5, 7,..., 224737 along the main diagonal and the number 1 in all the positions $a_{ij}$ with |*i*-*j*|=1,2,4,8,...,16384. What is the (1,1)-entry of $\mathbf{A}^{-1}$?"

In [ ]:
n = 20000
d = 2 .^ (0:14); d = [-d;d];
P = spdiags(primes(224737)',0,n,n);
B = spdiags(ones(n,length(d)),d,n,n);
A = P + B;
b = sparse(1,1,1,n,1);
x = pcg(A, b, 1e-15, 100, P); x(1)

**6.1. Radix-3 FFT.** The radix-3 FFT is similar to the [radix-2 FFT](#radix2fft). We'll verify that the code is correct by comparing it with a built-in FFT

In [ ]:
function y = fftx3(c)
  n = length(c);
  omega = exp(-2i*pi/n); 
  if mod(n,3) == 0   
    k = 0:n/3-1;
    u = [ fftx3(c(1:3:n-2)).';
        (omega.^k).*fftx3(c(2:3:n-1)).'; 
        (omega.^(2*k)).*fftx3(c(3:3:n)).'];
    F = exp(-2i*pi/3).^((0:2)'*(0:2));
    y = (F*u).'(:);
  else    
    F = omega.^([0:n-1]'*[0:n-1]);
    y = F*c;
  end
end

In [ ]:
v = rand(24,1);
[fft(v) fftx3(v)]

**6.2. Fast multiplication.** The following function uses FFTs to multiply two large integers (inputted as strings). We'll verify that the algorithm works by multiplying the [RSA-129 factors](https://en.wikipedia.org/wiki/RSA_numbers#RSA-129). 

In [ ]:
function [pq] = multiply(p,q)
  np  = [fliplr(p-'0') zeros([1 length(q)])];
  nq  = [fliplr(q-'0') zeros([1 length(p)])];
  pq = round(ifft(fft(np).*fft(nq)));
  carry = fix(pq/10);
  while sum(carry)>0, 
    pq = (pq - carry*10) + [0 carry(1:end-1)]; 
    carry = fix(pq/10);
  end
  n = max(find(pq));
  pq = char(fliplr(pq(1:n))+'0');
end

In [ ]:
multiply('32769132993266709549961988190834461413177642967992942539798288533',...
'3490529510847650949147849619903898133417764638493387843990820577')

**6.3. Fast discrete cosine transform.**

In [ ]:
function f = dct(f)
  n = size(f,1);
  w = exp(-0.5i*pi*(0:n-1)'/n);
  f = real(w.*fft(f([1:2:n  n-mod(n,2):-2:2],:)));
end

In [ ]:
function f = idct(f)
  n = size(f,1);
  f(1,:) = f(1,:)/2;
  w = exp(-0.5i*pi*(0:n-1)'/n);
  f([1:2:n  n-mod(n,2):-2:2],:) = 2*real(ifft(f./w));
end

In [ ]:
dct2 = @(f) dct(dct(f')');
idct2 = @(f) idct(idct(f')');

**6.4. DCT image compression.**

In [ ]:
pkg load signal 
function [B,A] = dctcompress2(A,d)
  n = size(A) ; n0 = floor(n*sqrt(d));
  B = dct2(A)(1:n0(1),1:n0(2));
  A = idct2(B,n);
end

In [ ]:
A = rgb2gray(double(imread([bucket "laura.png"])))/255;
[_,A0] = dctcompress2(A,0.01);
imshow(max(min([A,A0],1),0))

<a name="label18"></a>
## Numerical Analysis
**8.7. Aitken $\Delta^2$ process.** We approximate the series $$\sum_{n=0}^\infty \frac{(-1)^i}{2n+1} = \pi$$ using partial sums along with Aitken's extrapolation.

In [ ]:
aitken1 = @(x1,x2,x3) x3 - (x3-x2).^2./(x3 - 2*x2 + x1);
aitken2 = @(x1,x2,x3) (x1.*x3 - x2.^2)./(x3 - 2*x2 + x1);
n = 20000;
p = cumsum((-1).^[0:n]*4./(2*[0:n]+1));
p1 = aitken1(p(1:n-2),p(2:n-1),p(3:n));
p2 = aitken2(p(1:n-2),p(2:n-1),p(3:n));
loglog(abs(pi-p)); hold on; loglog(abs(pi-p2)); loglog(abs(pi-p1));

**8.12. Solving a nonlinear system.** We'll solve $$(x^2+y^2)^2 - 2 (x^2 - y^2) =0$$ $$(x^2+y^2 -1)^3-x^2y^3 = 0$$ using homotopy continuation and Newton's method.

In [ ]:
f = @(x,y) [(x^2+y^2)^2-2*(x^2-y^2); (x^2+y^2-1)^3-x^2*y^3];
df = @(x,y) [4*x*(x^2+y^2-1), 4*y*(x^2+y^2+1);
     6*x*(x^2+y^2 -1)^2-2*x*y^3, 6*y*(x^2+y^2 -1)^2-3*x^2*y^2];

In [ ]:
function x = homotopy(f,df,x)
  dxdt = @(t,x) -df(x(1),x(2))\f(x(1),x(2));
  [t,y] = ode45(dxdt,[0;1],x);
  x = y(end,:);
end

In [ ]:
function x = newton(f,df,x)
  for i = 1:100
    dx = -df(x(1),x(2))\f(x(1),x(2));
    x = x + dx;
    if norm(dx)<1e-8, return; end
  end 
end

In [ ]:
x0 = [-1;-1];
homotopy(f,df,x0)
newton(f,df,x0)

**9.2. Periodic parametric splines.** We modify the code [`spline_natural`](#spline_natural) (above) to  make a generate a spine with periodic boundary conditions. The function `evaluate_spline` is duplicated from the code above.

In [ ]:
function m = spline_periodic(x,y)
  h = diff(x);
  C = circshift(diag(h),[1 0]) + 2*diag(h+circshift(h,[1 0])) ...
     + circshift(diag(h),[0 1]);
  d = 6.*diff(diff([y(end-1);y])./[h(end);h]);
  m = C\d;  m = [m;m(1)];
end

In [ ]:
function [X,Y] = evaluate_spline(x,y,m,n)
  x = x(:); y = y(:); h = diff(x); 
  B = y(1:end-1) - m(1:end-1).*h.^2/6;
  A = diff(y)./h-h./6.*diff(m);
  X = linspace(min(x),max(x),n+1)';
  i = sum(x<=X');
  i(end) = length(x)-1; 
  Y = (m(i).*(x(i+1)-X).^3 + m(i+1).*(X-x(i)).^3)./(6*h(i)) ...
      + A(i).*(X-x(i)) + B(i);
end

In [ ]:
n = 10; nx = 20;
x = rand(n,1); y = rand(n,1);
x = [x;x(1)]; y = [y;y(1)];
t = [0;cumsum(sqrt(diff(x).^2+diff(y).^2))];
[_,X] = evaluate_spline(t,x,spline_periodic(t,x),nx*n);
[_,Y] = evaluate_spline(t,y,spline_periodic(t,y),nx*n);
plot(X,Y,x,y,'.','markersize',6);

**9.3. Radial basis functions.** Let's examine how a polynomial $y(x) = \sum_{i=0}^n c_i x^i$ compares with Gaussian and cubic radial basis functions $y(x) = \sum_{i=0}^n c_i \phi(x-x_i),$ taking $\phi(x)= \exp(-20x^2)$ and $\phi(x) = |x|^3$ an interpolant of the Heaviside function.

In [ ]:
n = 20; N = 200;
x = linspace(-1,1,n)';  X = linspace(-1,1,N)';
y = (x>0);

In [ ]:
phi1 = @(x,a) abs(x-a).^3;
phi2 = @(x,a) exp(-20*(x-a).^2);
phi3 = @(x,a) x.^a;
interp = @(phi,a) phi(X,a')*(phi(x,a')\y);

In [ ]:
Y1 = interp(phi1,x);
Y2 = interp(phi2,x);
Y3 = interp(phi3,(0:n-1)');
plot(x,y,X,Y1,X,Y2,X,Y3); ylim([-.5 1.5]);

**9.4. Collocation.** We'll use collocation to solve Bessel's equation.  We first define a function to solve general linear boundary value problems. And, then we define a function to interpolate between collocation points.

In [ ]:
function c = solve(L,f,bc,x)
  h = x(2)-x(1); n = length(x);
  S = ([1 -1/2 1/6; -2 0 2/3; 1 1/2 1/6]./[h^2 h 1])*L(x);
  A(2:n+1,1:n+2) = spdiags(S',[0 1 2],n,n+2);
  A(1,1:3) = [1/6 2/3 1/6];  
  A(n+2,n:n+2) = [1/6 2/3 1/6];
  d = [bc(1) f(x) bc(2)];
  c = A\d';
end

In [ ]:
function [X,Y] = build(c,x,N)
  X = linspace(x(1),x(end),N);
  h = x(2) - x(1);
  i = floor(X/h)+1; i(N) = i(N-1);
  C = [c(i) c(i+1) c(i+2) c(i+3)]';
  B = @(x)  [(1-x).^3;4-3*(2-x).*x.^2;4-3*(1+x).*(1-x).^2;x.^3]/6;
  Y = sum(C.*B((X-x(i))/h));
end

Now, we can solve the Bessel equation $xu''+u'+xu =0$ with boundary conditions $u(0)=1$ and $u(b)=0$.

In [ ]:
n = 15; N = 141
L = @(x) [x;ones(size(x));x];
f = @(x) zeros(size(x));
b = fzero(@(z) besselj(0, z), 11);
x = linspace(0,b,n); 
c = solve(L,f,[1,0],x);
[X,Y] = build(c,x,N);
plot(X,Y,X,besselj(0,X))

Finally, we'll explore the error and convergence rate.

In [ ]:
n = 10*2.^(1:6);
for i = 1:length(n)
  x = linspace(0,b,n(i)); 
  c = solve(L,f,[1,0],x);
  [X,Y] = build(c,x,n(i));
  e(i) = sqrt(sum((Y-besselj(0,X)).^2)/n(i));
end
loglog(n,e,'.-');
s = polyfit(log(n),log(e),1);
printf("slope: %f",s(1))

**10.3. Fractional derivatives.** We'll plot the fractional derivatives for a function.

In [ ]:
n = 128; L = 2; 
x = (0:n-1)/n*L-L/2;
k = [0:(n/2-1) (-n/2):-1]*(2*pi/L);
f = exp(-6*x.^2);
for p = 0:0.1:1
  d = real(ifft((1i*k).^p.*fft(f)));
  plot(x,d,'m'); hold on;
end

**10.4. Handwriting classification.** We'll use  to train a convolutional neural net using MNIST data and then test the model.

**11.1. Finite difference approximation.**  Let's find coefficients to the third-order approximation of $f'(x)$ for nodes at $x$, $x+h$, $x+2h$ and $x+3h$. 

In [ ]:
d = [0,1,2,3]; n = length(d);
V = fliplr(vander(d)) ./ factorial([0:n-1]);
coeffs = inv(V);
trunc = coeffs*d'.^n/factorial(n);

The coefficients of the finite difference approximation of the derivative and coefficient of the truncation error are given by

In [ ]:
display(["coefficients: " rats(coeffs(2,:))])
display(["truncation: " rats(trunc(2))])

**11.2. Richardson extrapolation.** The following code is an iterative version of the recursive [`richardson`](#richardson_extrapolation) function above:

In [ ]:
function a = richardson(f,x,m)
  for i=1:m 
    D(i) =  phi(f,x,2^i); 
    for j = i-1:-1:1 
      D(j) = (4^(i-j)*D(j+1) - D(j))/(4^(i-j) - 1);
    end
  end
  a = D(1);
end

**11.3. Automatic differentiation.** Let's extend the [`Dual class`](#dualclass) above by adding methods for division, cosine, and square root to the class definition. We'll also add a few more help functions. You'll first need to save the following code as an m-file called `dual.m`. I've added a Jupyter magic command to the top of the cell to do this.

In [ ]:
%%file dual.m
classdef dual
  properties
    value
    deriv
  end 
  methods
    function obj = dual(a,b)
      obj.value = a;
      obj.deriv = b;
    end
    function h = plus(u,v)
      if ~isa(u,'dual'), u = dual(u,0); end
      if ~isa(v,'dual'), v = dual(v,0); end
      h = dual(u.value + v.value, u.deriv + v.deriv);
    end
    function h = mtimes(u,v)
      if ~isa(u,'dual'), u = dual(u,0); end
      if ~isa(v,'dual'), v = dual(v,0); end
      h = dual(u.value*v.value, u.deriv*v.value + u.value*v.deriv);
    end
    function h = sin(u)
      h = dual(sin(u.value), cos(u.value)*u.deriv);
    end
    function h = minus(u,v)
      h = u + (-1)*v;
    end
    function h = mrdivide(u,v)
     if ~isa(u,'dual'), u = dual(u,0); end
     if ~isa(v,'dual'), v = dual(v,0); end
     h = dual(u.value/v.value, ...
       (v.value*u.deriv-1*u.value*v.deriv)/(v.value*v.value));
    end
    function h = cos(u)
      h = dual(cos(u.value), -1*sin(u.value)*u.deriv);
    end
    function h = sqrt(u)
      h = dual(sqrt(u.value), u.deriv/(2*sqrt(u.value)));
    end
  end
end

In [ ]:
clear functions

Now, we can define Newton's method using this new Dual class and use it to find the zero of $4\sin x + \sqrt{x}$.

In [ ]:
function x = get_zero(f,x)
  tolerance = 1e-12; delta = 1;
  while abs(delta)>tolerance,
    fx = f(dual(x,1));
    delta = fx.value/fx.deriv;
    x -= delta;
  end
end

In [ ]:
get_zero(@(x) 4*sin(x) + sqrt(x), 4)

We can find a minimum or maximum of $4\sin x + \sqrt{x}$ by modifying Newton's method.

In [ ]:
function x = get_extremum(f,x)
  tolerance = 1e-12; delta = 1;
  while abs(delta)>tolerance,
    fx = f(dual(dual(x,1),1));
    delta = fx.value.deriv/fx.deriv.deriv;
    x -= delta;
  end
end

In [ ]:
get_extremum(@(x) 4*sin(x) + sqrt(x), 4)

**11.4. Cauchy differentiation formula.**

In [ ]:
function d = cauchyderivative(f, a, p, n = 20, r = 0.1)
  w = exp(2*pi*1i*(0:(n-1))/n);
  d = factorial(p)/(n*r^p)*sum(f(a+r*w)./w.^p)
end

In [ ]:
f = @(x) exp(x)/(cos(x).^3 + sin(x).^3)
cauchyderivative(f, 0, 6)

**11.5. Gauss–Legendre quadrature.** The following  function computes the nodes and weights for  Gauss–Legendre quadrature by using Newton's method to find the roots of $\mathrm{P_n}(x)$. We'll verify the function on a toy problem.

In [ ]:
function [x,w] = gauss_legendre(n)
  x = -cos((4*(1:n)-1)*pi/(4*n+2))';
  dx = ones(n,1);
  dP = 0;
  while(max(abs(dx))>1e-16),
    P = [x ones(n,1)];
    for k = 2:n
      P = [((2*k - 1)*x.*P(:,1)-(k-1)*P(:,2))/k, P(:,1)];
    end
    dP = n*(x.*P(:,1) - P(:,2))./(x.^2-1);
    dx =  P(:,1) ./ dP(:,1);
    x = x - dx;
  end
  w = 2./((1-x.^2).*dP(:,1).^2);
end

In [ ]:
f = @(x) 2*sqrt(1-x.^2);
[x,w] = gauss_legendre(10);
w'*f(x)

**11.7. Fundamental solution to the heat equation.** We'll use Gauss–Hermite quadrature to compute the solution to the heat equation $$u(t,x) = \frac{1}{\sqrt{4\pi t}}\int_{-\infty}^\infty  u_0(s) \mathrm{e}^{-(x-s)^2/4t} \;\mathrm{d}s.$$

In [ ]:
function [nodes,weights] = gauss_hermite(n)
  b = (1:n-1)/2;
  a = zeros(n,1);
  scaling = sqrt(pi);
  [v,s] = eig(diag(sqrt(b),1) + diag(a) + diag(sqrt(b),-1));
  weights = scaling*v(1,:).^2;
  nodes = diag(s);
end

In [ ]:
[s,w] = gauss_hermite(20);
u0 = @(x) sin(x);
u = @(t,x) w * u0(x-2*sqrt(t)*s)/sqrt(pi);
x = linspace(-12,12,100);
plot(x,u(1,x))

**11.8. Monte Carlo integration.** The following  function the volume of an $d$-dimensional sphere using $n$ samples and $m$ trials. We'll use it to verify that error of Monte Carlo integration is $O(1/\sqrt{n})$.

In [ ]:
mc_pi = @(n,d,m) sum(sum(rand(d,n,m).^2,1)<1)./n*2^d;

In [ ]:
m = 20; error = []; N = 2 .^ (1:20);
for n = N
  error = [error sum(abs(pi - mc_pi(n,2,m)))/m];
end
s = polyfit(log(N),log(error),1);
printf("slope: %f",s(1))
loglog(N,exp(s(2)).*N.^s(1),N,error,'.');

<a name="label19"></a>
## Numerical Differential Equations
**12.4. Runge–Kutta  stability.** The following code plots the region of absolute stability for a Runge–Kutta method with tableau `A` and `b`.

In [ ]:
A = [0   0   0   0   0,
     1/3 0   0   0   0,
     1/6 1/6 0   0   0,
     1/8 0   3/8 0   0,
     1/2 0  -3/2 2   0];
b = [1/6 0   0   2/3 1/6];

In [ ]:
n = length(b);
N = 100;
x = linspace(-4,4); y = x'; r = zeros(N,N);
lk = x + 1i*y;
E = ones(n,1);
for i = 1:N, for j=1:N 
  r(i,j) = 1+ lk(i,j) * b*(( eye(n) - lk(i,j)*A)\E);
end, end
contour(x,y,abs(r),[1 1],'k');
axis([-4 4 -4 4]);

**12.7. Third-order IMEX coefficients.** We can determine the coefficients of a third-order IMEX method by inverting a system of linear equations.

In [ ]:
i = (0:3)';  
a = ((-(i+1)').^i./factorial(i))\[1;0;0;0];
b = ((-i').^i./factorial(i))\[0;1;0;0];

In [ ]:
display(["RHS:" rats(a')])
display(["LHS:" rats(b')])

**12.8. Predictor-corrector stability.** We'll use the  [`multistepcoefficients`](#multistepcoefficients) introduced earlier. The following function provides the orbit of points in the complex plane for an $n$th order  Adams–Bashforth–Moulton PE(CE)$^m$.

In [ ]:
function [a,b] = multistepcoefficients(m,n)
  s = length(m) + length(n) - 1;
  A = (m+1).^((0:s-1)');
  B = ((0:s-1).*(n+1)'.^[0,0:s-2])';
  c = -[A(:,2:end) B]\ones(s,1);
  a = [1;c(1:length(m)-1)];
  b = [c(length(m):end)];
end

In [ ]:
function z = PECE(n,m)
  [_,a] = multistepcoefficients([0,1],1:n-1);
  [_,b] = multistepcoefficients([0,1],0:n-1);
  for i = 1:200
    r =  exp(2i*pi*(i/200));
    c(1) = r - 1;
    c(2:m+1) = r + r.^(1:n-1)*b(2:end)'/b(1);
    c(m+2) = r.^(1:n-1)*a/b(1);
    z(i,:) = roots(fliplr(c))'/b(1);
  end
end

In [ ]:
for i= 1:4 
  plot(PECE(2,i),'.k'); hold on; axis equal
end

**12.9. Padé approximant.** We'll build a function to compute the coefficients of the Padé approximant to $log(r)$.

In [ ]:
function [p,q] = pade(a,m,n)
  A = eye(m+n+1);
  for i=1:n, A(i+1:end,m+1+i) = -a(1:m+n+1-i); end
  pq = A\a(1:m+n+1);
  p = pq(1:m+1); q = [1; pq(m+2:end)];
end

In [ ]:
m = 3; n = 2;
a = [0 ((-1).^(0:m+n)./(1:m+n+1))]';
[p,q] = pade(a,m,n)

In [ ]:
S = @(n) inv(pascal(n,-1)');
S(m+1)*p, S(n+1)*q

**12.13. SIR solution.** Let's solve the susceptible-infected-recovered (SIR) model for infectious diseases using a general ODE solver.

In [ ]:
SIR = @(t,y,b,g) [-b*y(1)*y(2);b*y(1)*y(2)-g*y(2);g*y(2)];
tspan = linspace(0,15,100); y0 = [0.99, 0.01, 0];
[t,y] = ode45(@(t,y) SIR(t,y,2,0.4),tspan,y0);
plot(t,y(:,1),t,y(:,2),t,y(:,3));

**12.14. Duffing equation.** We'll use a high-order, explicit ODE solver for the Duffing equation.

In [ ]:
duffing = @(t,x,g) [x(2); -g*x(2)+x(1)-x(1).^3+0.3*cos(t)]; 
tspan = linspace(0,200,2000);
[t,x] = ode45(@(t,x) duffing(t,x,0.37), tspan, [1,0]);
plot(x(:,1),x(:,2));

**12.15. Shooting method.** We'll solve the Airy equation $y'' - xy = 0$ using the shooting method that incorporates an initial value solver into a nonlinear root finder.

In [ ]:
function error = shooting(x,f,xspan,bc)
  [t,y] = ode45(f,xspan,[bc(1),x]);
  error = y(end,1)- bc(2);
end

In [ ]:
xspan = [-12, 0]; bc = [1, 1]; guess = 5;
airy = @(x,y) [y(2);x*y(1)];
v = fsolve(@(x) shooting(x,airy,xspan,bc),guess)

Once we have our second initial value, we can plot the solution:

In [ ]:
[t,y] = ode45(airy,linspace(-12,0,200),[bc(1),v]);
plot(t,y(:,1))

**13.5. Dufort–Frankel method.** We'll use the Dufort–Frankel method to solve the heat equation. While this method is unconditionally stable, it generates the wrong solution. Notice that while the long-term behavior is dissipative, the solution is largely oscillatory, and the dynamics are more characteristic of a viscous fluid than heat propagation.(Plotting the solution as an SVG may result in an error. In this case, we'll display it as a PNG.

In [ ]:
dx = 0.01; dt = 0.01;
L = 1; x = (-L:dx:L)'; m = length(x);
V = exp(-8*x.^2); U = V;
c = dt/dx^2; a = 0.5 + c; b = 0.5 - c;
B = c*spdiags([ones(m,1),zeros(m,1),ones(m,1)],-1:1,m,m);
B(1,2) = B(1,2)*2; B(end,end-1) = B(end,end-1)*2;
for i = 1:420,
  if mod(i,3)==1, area(x, (V-i/300),-1,'facecolor','w'); hold on; end
  Vo = V; V = (B*V+b*U)/a; U = Vo;
end
ylim([-1,1]); set(gca,'xtick',[],'ytick',[])

**13.7. Schrödinger equation.** Let's solve the Schrödinger equation for a harmonic potential using the Strang splitting Crank–Nicolson and confirm that the method is $O(h^2,k^2)$.

In [ ]:
function psi = schroedinger(n,m,eps)
  x = linspace(-4,4,n)'; dx = x(2)-x(1); dt = 2*pi/m; V = x.^2/2;
  psi = exp(-(x-1).^2/(2*eps))/(pi*eps)^(1/4);
  diags = repmat([1 -2 1],[n 1])/dx^2;
  D = 0.5i*eps*spdiags(diags,-1:1,n,n) - 1i/eps*spdiags(V,0,n,n);
  D(1,2) = 2*D(1,2); D(end,end-1) = 2*D(end,end-1); 
  A = speye(n) + (dt/2)*D; 
  B = speye(n) - (dt/2)*D;  
  for i = 1:m
    psi = B\(A*psi);
  end
end

We'll loop over several values for time steps and mesh sizes and plot the error. This may take a while. Go get a snack.

In [ ]:
eps = 0.3; m = 20000; n = floor(logspace(2,3.7,6));
x = linspace(-4,4,m)';
psi_m = -exp(-(x-1).^2/(2*eps))/(pi*eps)^(1/4);
for i = 1:length(n), 
  x = linspace(-4,4,n(i))'; 
  psi_n = -exp(-(x-1).^2/(2*eps))/(pi*eps)^(1/4);
  error_t(i) = norm(psi_m - schroedinger(m,n(i),eps))/m;
  error_x(i) = norm(psi_n - schroedinger(n(i),m,eps))/n(i);
end
loglog(2*pi./n,error_t,'.-r',8./n,error_x,'.-k');

**13.8. Polar heat equation.** We'll solve a radially symmetric heat equation. Although we divide by zero at $r=0$ when constructing the Laplacian operator, we subsequently overwrite the resulting  `inf` term when we apply the boundary condition.

In [ ]:
T = 0.5; m = 100; n = 100;
r = linspace(0,2,m)'; dr = r(2)-r(1); dt = T/n;
u = tanh(32*(1-r));
tridiag = [1 -2 1]/dr^2  + (1./r).*[-1 0 1]/(2*dr);
D = spdiags(tridiag,-1:1,m,m)';
D(1,1:2) = [-4 4]/dr^2; D(m,m-1:m) = [2 -2]/dr^2; 
A = speye(m) - 0.5*dt*D; 
B = speye(m) + 0.5*dt*D;  
for i = 1:n
  u = A\(B*u);
end
area(r,u,-1,"edgecolor",[1 .5 .5],"facecolor",[1 .8 .8]);

**13.9. Open boundaries.** We can approximate open boundaries by spacing the grid points using a sigmoid function such as $\mathrm{arctanh}\, x$. We start by defining a function `logitspace`, a logit analog to `np.linspace`. Then we define a Laplacian operator using arbitrary grid spacing. Finally, we solve the heat equation using the Crank–Nicolson using both equally-spaced and logit-spaced grid points.

In [ ]:
logitspace = @(x,n,p) x*atanh(linspace(-p,p,n)')/atanh(p);

In [ ]:
function D = laplacian(x)
  h = diff(x); h1 = h(1:end-1); h2 = h(2:end); n = length(x);
  diags = 2./[h1(1).^2, -h1(1).^2, 0;
  h2.*(h1+h2), -h1.*h2, h1.*(h1+h2);
  0, -h2(end).^2, h2(end).^2];
  D = spdiags(diags,-1:1,n,n)';
end

In [ ]:
function u =  heat_equation(x,t,u)
  n = 40; dt = t/n;
  D = laplacian(x);
  A = speye(n) - 0.5*dt*D;
  B = speye(n) + 0.5*dt*D;
  for i = 1:n
    u = A\(B*u);
  end
end

In [ ]:
phi = @(x,t,s) exp(-s*x.^2/(1+4*s*t))/sqrt(1+4*s*t);
t = 15; m = 40;
x = logitspace(20,m,.999);
u = heat_equation(x,t,phi(x,0,10));
plot(x,u,'.-',x,phi(x,t,10),'k')

**13.10. Allen–Cahn equation.** We'll solve the Allen–Cahn equation using  Strang splitting. 

In [ ]:
L = 16; m = 400; dx = L/m;
T = 4; n = 1600; dt = T/n;
x = linspace(-L/2,L/2,m);
u = tanh(x.^4 - 16*(2*x.^2-x'.^2));
D = spdiags(repmat([1 -2 1],[m 1]),-1:1,m,m)/dx^2;
D(1,2) = 2*D(1,2); D(end,end-1) = 2*D(end,end-1);
A = speye(m) + 0.5*dt*D;
B = speye(m) - 0.5*dt*D; 
f = @(u,dt) u./sqrt(u.^2 - (u.^2-1).*exp(-50*dt));
u = f(u,dt/2);
for i = 1:n
  u = (B\(A*(B\(A*u))'))';
  if i<n, u = f(u,dt); end
end
u = f(u,dt/2);

In [ ]:
image((u+1)/2*100);colormap(gray(100));

**14.7. Burgers' equation.**

In [ ]:
m = 100; x = linspace(-1,3,m); dx = x(2)-x(1);
n = 100; Lt = 4; dt = Lt/n;
lambda = dt/dx;
f = @(u) u.^2/2; fp = @(u) u;
u = (x>=0)&(x<=1); 
for i = 1:n
  fu = f([u(1) u]); fpu = fp([u(1) u]);
  a = max(abs(fu(1:n-1)),abs(fu(2:n)));
  F = (fu(1:n-1)+fu(2:n))/2 - a.*diff(u)/2;
  u = u - lambda*(diff([0 F 0]));
end
area(x,u,"edgecolor",[.3 .5 1],"facecolor",[.6 .8 1]);

**14.8. Dam break problem.**

In [ ]:
function s = slope(u)
  limiter = @(t) (abs(t)+t)./(1+abs(t));  
  du = diff(u);
  s = [[0 0];du(2:end,:).*limiter(du(1:end-1,:)./(du(2:end,:) ...
    + (du(2:end,:)==0)));[0 0]];
end

In [ ]:
F = @(u) [u(:,1).*u(:,2), u(:,1).*u(:,2).^2+0.5*u(:,1).^2];
m = 1000; x = linspace(-.5,.5,m)'; dx = x(2)-x(1);
T = 0.25; n = ceil(T/(dx/2)); dt = (T/n)/2;  c = dt/dx;
u = [0.8*(x<0)+0.2,0*x]; 
j = 1:m-1; 
for i = 1:n
  v = u-0.5*c*slope(F(u));
  u(j+1,:)=(u(j,:)+u(j+1,:))/2 - diff(slope(u))/8-c*diff(F(v)); 
  v = u-0.5*c*slope(F(u));
  u(j,:)=(u(j,:)+u(j+1,:))/2 - diff(slope(u))/8-c*diff(F(v));
end
plot(x,u(:,1));

**15.1. Finite element method.**

In [ ]:
m=10;
x=linspace(0,1,m)'; h=x(2)-x(1);
A=diag(repmat(-1/h-h/6,[m-1 1]),-1)+diag(repmat(1/h-h/3,[m 1]));
A = A + A'; A(1,1)=A(1,1)/2; A(m,m)=A(m,m)/2;
b=[-2/3*h^3;-4/3*h^3-8*h*x(2:m-1).^2;-4*h+8*h^2/3-2*h^3/3+1];
u=A\b;
s=(-16)+8.*x.^2+15.*cos(x).*csc(1);
plot(x,s,'o-',x,u,'.-');

**15.2. Finite element method.**

In [ ]:
m = 8;
x = linspace(0,1,m+2); h = x(2)-x(1);
D = @(a,b,c) (diag(a*ones(m-1,1),-1) + ... 
   diag(b*ones(m,1),0) + diag(c*ones(m-1,1),1))/h^3;
M = [D(-12,24,-12) D(-6,0,6);D(6,0,-6) D(2,8,2)];
b = [ones([m 1])*h*384;zeros([m 1])];
u = M\b;
plot(x,16*(x.^4 - 2*x.^3 + x.^2),'o-',x,[0;u(1:m);0],'.-');

**16.2. Burgers' equation.** Fourier spectral methods perform poorly on problems that develop discontinuities such as Burgers' equation.  Gibbs oscillations develop around the discontinuity, and these oscillations will spread and grow because Burgers' equation is dispersive. Ultimately, the oscillations overwhelm the solution.

In [ ]:
m = 128; x = (1:m)'/m*(2*pi)-pi;
k = 1i*[0:m/2 -m/2+1:-1]';
f = @(t,u) -(ifft(k.*fft(0.5*u.^2)));
[t,u] = ode45(f,[0 1.5],exp(-x.^2)); 
plot(x,u(end,:))

**16.3. KdV equation.** We'll solve the KdV equation using integrating factors. We first set the initial conditions and parameters. Then, we define the integrating factor `G` and the right-hand side `f` of the differential equation. Finally, we animate the solution. Notice the two soliton solution.

In [ ]:
phi = @(x,x0,c) 0.5*c*sech(sqrt(c)/2*(x-x0)).^2;
L = 30; T = 1.0; m = 256;
x = (1:m)'/m*L-L/2;
k = 1i*[0:(m/2) (-m/2+1):-1]'*(2*pi/L);

In [ ]:
G = @(t) exp(-k.^3*t);
f = @(t,w) -G(t).\(3*k.*fft(ifft(G(t).*w).^2));

In [ ]:
u = phi(x,-4,4) + phi(x,-9,9);
w = fft(u);
[t,w] = ode45(f,linspace(0,T,40),w);
u = real(ifft(G(t').*w.').');

In [ ]:
for i=40:-1:1, 
  area(x, T*(u(i,:)+i)/40,'facecolor','w'); hold on; 
end

**16.4. Swift–Hohenberg equation.** We'll use Strang splitting to solve the  Swift–Hohenberg equation.

In [ ]:
eps = 1; m = 256; L = 100; n = 2000; dt=100/n;
U = (rand(m)>.5)-0.5;
colormap(gray(256))
k = [0:(m/2) (-m/2+1):-1]*(2*pi/L);
D2 = (1i*k).^2+(1i*k').^2;
E = exp(-(D2+1).^2*dt);
f = @(U) U./sqrt(U.^2/eps + exp(-dt*eps)*(1-U.^2/eps));
for i=1:n
   U = f(ifft2(E.*fft2(f(U))));
end 
imshow(real(U))